# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing scikit learn functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    

In [72]:
import math
import string
import numpy as np
import pandas as pd
import os

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [73]:
os.getcwd()
products = pd.read_csv('./data/amazon_baby.csv')
print('No. of records in products dataframe: {}'.format(len(products.index)))

No. of records in products dataframe: 183531


Now, let us see a preview of what the dataset looks like.

In [74]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [75]:
products.loc[269, :]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

IMPORTANT . Make sure to fill n/a values in the review column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the review columns as follows:

In [76]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [77]:
def remove_punctuation(text):
    import string    
    return text.translate(str.maketrans('', '', string.punctuation))

products['review'] = products.review.astype(str)
products["review_without_punctuation"] = products['review'].apply(remove_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [78]:
def word_count(sentence):
    word_count_dict = {}
    for word in [aword.lower() for aword in sentence.split()]:
        if word in word_count_dict.keys():
            word_count_dict[word] = word_count_dict[word] + 1
        else:
            word_count_dict[word] = 1
    return word_count_dict

products["word_count"] = products["review_without_punctuation"].apply(word_count)
products.loc[269, :]['word_count']

{'a': 1, 'favorite': 1, 'in': 1, 'our': 1, 'house': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [79]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [80]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.head()

,name,review,rating,review_without_punctuation,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{'when': 2, 'the': 6, 'binky': 3, 'fairy': 3, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [81]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(products, test_size=0.2, random_state=42)
print(len(train_data))
print(len(test_data))
train_y = train_data['sentiment']
test_y = test_data['sentiment']

133401
33351


## Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features . Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

1. Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
2. Compute the occurrences of the words in each review and collect them into a row vector.
3. Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix .
4. Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix .

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
vectorizer.fit(train_data['review_without_punctuation'])
# input feature matrix for training set
train_X = vectorizer.transform(train_data['review_without_punctuation'])
# input feature matrix for test set
test_X = vectorizer.transform(test_data['review_without_punctuation'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data.

Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix ( train_X ) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model .

In [83]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(train_X, train_y)

C:\Users\anupam\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\anupam\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [84]:
sentiment_model.coef_

array([[-6.32054538e-01,  5.27842210e-04,  1.27606062e-02, ...,
         7.06640848e-03,  5.00426878e-03, -9.97085646e-05]])

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [109]:
def print_model_coeff(model):
    coeffs = model.coef_
    num_coeff = coeffs.shape[1]
    print('No. of coefficients: {}'.format(num_coeff))
    num_positive_coeff = [coeff for coeff in coeffs[0] if coeff > 0]
    print('No of +ve coefficients: {}'.format(len(num_positive_coeff)))
    print('No of -ve coefficients: {}'.format(num_coeff - len(num_positive_coeff)))
    
print_model_coeff(sentiment_model)    

No. of coefficients: 121841
No of +ve coefficients: 85784
No of -ve coefficients: 36057


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [87]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

80054    2
44765    5
48173    5
Name: rating, dtype: int64


,name,review,rating,review_without_punctuation,word_count,sentiment
80054,"Simple Wishes Hands-Free Breastpump Bra, Pink,...",I like the idea but the problems:-awkward clos...,2,I like the idea but the problemsawkward closin...,"{'i': 1, 'like': 1, 'the': 13, 'idea': 1, 'but...",-1
44765,"Moby Wrap UV SPF 50+ 100% Cotton Baby Carrier,...","This is my 2nd Moby, just wanted another color...",5,This is my 2nd Moby just wanted another color ...,"{'this': 3, 'is': 4, 'my': 7, '2nd': 1, 'moby'...",1
48173,The First Years American Red Cross Deluxe Nail...,This is the best nail clipper! Definitely reco...,5,This is the best nail clipper Definitely recom...,"{'this': 2, 'is': 1, 'the': 3, 'best': 1, 'nai...",1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [88]:
sample_test_data.iloc[0]['review']

"I like the idea but the problems:-awkward closing! zipper requires two hands but you are likely to have the cups of the pump fall out or slip around while zipping-size range XS-medium?? got to be kidding. it's very picky trying to get the width of the back and front so that your nipples fall right in the center of the front nipple openings. and if they're off to the side the cups of the pump slide and nipples rub the insides during pumping-- OUCH!-not worth the price-not a bra I'd want to wear all day and getting it on and off also not convenient."

That review seems negative.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (+1), the review is positive.

In [89]:
sample_test_data.iloc[1]['review']

"This is my 2nd Moby, just wanted another color to switch off with.  Thought it would be nice to keep one in the house and one in the car. However, just had to pass one of mine on to a friend who tried my moby out and loved it so much!  I guess I'll have to get yet another!  Moby wraps are excellent for your back and shoulders, and the child loves being close to mommy or whomever is wearing the wrap.  I even had my almost 3 year old daughter in it a couple weeks ago, although I use it most with my newborn, and will continue to as she grows.  This is my 3rd baby using Moby's with- I love them!  They are so comfortable and are simple to put on after following the instructions and doing it a few times.  I wear it going out sometimes, so it's convenient to just slip the baby in when I arrive at my destination.  Breathable cotton is great and it has a little bit of a stretch to it.  It's also nice to be able to face the baby outward once they can hold their heads up really well.  Mine have 

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in sample_test_data . In scikit-learn, you can call the decision_function() function.

Hint : You'd probably need to convert sample_test_data into the sparse matrix format first..

In [90]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_without_punctuation'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[-0.54702071 17.02641369  4.88924495]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate predicted labels for sample_test_data .

Checkpoint : Make sure your class predictions match with the ones obtained from sentiment_model . The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

In [91]:
predicted_sentiment = sentiment_model.predict(sample_test_matrix)
print(predicted_sentiment)

[-1  1  1]


### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [92]:
probability_calculated = 1 / (1 + np.exp(-scores))
print(probability_calculated)

[0.3665559  0.99999996 0.99252913]


**Checkpoint**: Make sure your probability predictions match the ones obtained from sentiment_model.

In [93]:
print("Class predictions according to sklearn:" )
predicted_probab = sentiment_model.predict_proba(sample_test_matrix)
print(predicted_probab[:, 1])
sample_test_data["predicted_probab"] = predicted_probab[:, 1]
sample_test_data.sort_values(by=['predicted_probab'])

Class predictions according to sklearn:
[0.3665559  0.99999996 0.99252913]


C:\Users\anupam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,name,review,rating,review_without_punctuation,word_count,sentiment,predicted_probab
80054,"Simple Wishes Hands-Free Breastpump Bra, Pink,...",I like the idea but the problems:-awkward clos...,2,I like the idea but the problemsawkward closin...,"{'i': 1, 'like': 1, 'the': 13, 'idea': 1, 'but...",-1,0.366556
48173,The First Years American Red Cross Deluxe Nail...,This is the best nail clipper! Definitely reco...,5,This is the best nail clipper Definitely recom...,"{'this': 2, 'is': 1, 'the': 3, 'best': 1, 'nai...",1,0.992529
44765,"Moby Wrap UV SPF 50+ 100% Cotton Baby Carrier,...","This is my 2nd Moby, just wanted another color...",5,This is my 2nd Moby just wanted another color ...,"{'this': 3, 'is': 4, 'my': 7, '2nd': 1, 'moby'...",1,1.000000


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use sklearn to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. 
2.  Sort the data according to those predictions and pick the top 20. 

In [94]:
test_predicted_probab = sentiment_model.predict_proba(test_X)
#tpp = sorted(test_predicted_probab[:, 1], reverse=True)
#print(tpp)
test_data["predicted_probab"] = test_predicted_probab[:, 1]
sorted_test_data = test_data.sort_values(by=['predicted_probab'], ascending=False)
sorted_test_data.head(n=20)

C:\Users\anupam\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,name,review,rating,review_without_punctuation,word_count,sentiment,predicted_probab
59320,Evenflo Aura Select Travel System - Caroline,"I'm about to be a first-time mom, so I spent w...",5,Im about to be a firsttime mom so I spent week...,"{'im': 2, 'about': 3, 'to': 14, 'be': 4, 'a': ...",1,1.0
142686,"Thirsties Duo Diaper Snap, Mud, Size One (6-18...",I will try to keep this short. Have tried man...,5,I will try to keep this short Have tried many...,"{'i': 18, 'will': 2, 'try': 1, 'to': 15, 'keep...",1,1.0
123632,"Zooper 2011 Waltz Standard Stroller, Flax Brown",I did a TON of research before I purchased thi...,5,I did a TON of research before I purchased thi...,"{'i': 20, 'did': 1, 'a': 52, 'ton': 1, 'of': 3...",1,1.0
50735,"Joovy Zoom 360 Swivel Wheel Jogging Stroller, ...",The joovy zoom 360 was the perfect solution fo...,5,The joovy zoom 360 was the perfect solution fo...,"{'the': 61, 'joovy': 1, 'zoom': 1, '360': 1, '...",1,1.0
129722,Bumbleride 2011 Flite Lightweight Compact Trav...,This is a review of the 2012 Bumbleride Flite ...,5,This is a review of the 2012 Bumbleride Flite ...,"{'this': 12, 'is': 23, 'a': 38, 'review': 2, '...",1,1.0
100166,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,"{'i': 28, 'bought': 1, 'this': 6, 'carrier': 7...",1,1.0
168086,Buttons Cloth Diaper Cover - One Size - 8 Colo...,Buttons vs. Best Bottoms reviewFirst thing I w...,5,Buttons vs Best Bottoms reviewFirst thing I wa...,"{'buttons': 15, 'vs': 1, 'best': 19, 'bottoms'...",1,1.0
135152,Maxi-Cosi Pria 70 with Tiny Fit Convertible Ca...,We've been using Britax for our boy (now 14 mo...,5,Weve been using Britax for our boy now 14 mont...,"{'weve': 2, 'been': 5, 'using': 1, 'britax': 5...",1,1.0
73725,"Chicco Cortina Keyfit 30 Travel System, Miro",UPDATE 11/20/13 - I went ahead and used a tiny...,4,UPDATE 112013 I went ahead and used a tiny bi...,"{'update': 2, '112013': 1, 'i': 54, 'went': 2,...",1,1.0
57108,BabyPlus Prenatal Education System,I started wearing the Babyplus when I was 18 w...,5,I started wearing the Babyplus when I was 18 w...,"{'i': 16, 'started': 2, 'wearing': 1, 'the': 1...",1,1.0


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [95]:
sortasc_test_data = test_data.sort_values(by=['predicted_probab'])
sortasc_test_data.head(n=20)

,name,review,rating,review_without_punctuation,word_count,sentiment,predicted_probab
147902,Graco Pack 'n Play Playard - Dempsey,My disappointment with this product prompted m...,1,My disappointment with this product prompted m...,"{'my': 5, 'disappointment': 2, 'with': 11, 'th...",-1,1.519005e-20
175191,"Zooper Twist Escape Stroller, Summer Day",I had to return this stroller for three reason...,1,I had to return this stroller for three reason...,"{'i': 37, 'had': 4, 'to': 21, 'return': 2, 'th...",-1,1.536451e-18
89902,"Peg-Perego Aria Twin Stroller, Java",I am so incredibly disappointed with the strol...,1,I am so incredibly disappointed with the strol...,"{'i': 16, 'am': 1, 'so': 3, 'incredibly': 1, '...",-1,4.743083e-16
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,"{'i': 18, 'thought': 1, 'it': 8, 'sounded': 1,...",-1,2.919969e-14
57234,"Dream On Me Bassinet, Blue",My husband and I are VERY disappointed and sho...,1,My husband and I are VERY disappointed and sho...,"{'my': 3, 'husband': 2, 'and': 14, 'i': 12, 'a...",-1,5.031679e-14
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,"{'this': 12, 'is': 15, 'my': 6, 'second': 3, '...",-1,1.477205e-13
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,"{'i': 26, 'will': 4, 'try': 2, 'to': 29, 'writ...",-1,4.387718e-13
27310,Evenflo Expressions Plus High Chair - 3's Company,PLEASE HEED THE OTHER REVIEWERS WARNINGS ON TH...,1,PLEASE HEED THE OTHER REVIEWERS WARNINGS ON TH...,"{'please': 2, 'heed': 1, 'the': 18, 'other': 2...",-1,2.586368e-12
145109,"Withings Smart Baby Monitor, White","Ok, the good. During the day, the quality of ...",1,Ok the good During the day the quality of the...,"{'ok': 1, 'the': 31, 'good': 1, 'during': 1, '...",-1,4.937239e-12
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,"{'do': 1, 'not': 2, 'buy': 1, 'this': 6, 'baby...",-1,1.550801e-11


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [96]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    predicted_labels = model.predict(data)
    # Compute the number of correctly classified examples
    is_correct = predicted_labels == true_labels
    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = len(predicted_labels[is_correct]) / len(is_correct)    
    return round(accuracy, 2)

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [97]:
get_classification_accuracy(sentiment_model, test_X, test_y)

0.93

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [98]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [99]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [100]:
vectorizer_subset = CountVectorizer(vocabulary=significant_words, token_pattern=r'\b\w+\b')
vectorizer_subset.fit(train_data['review_without_punctuation'])
train_subsetX = vectorizer_subset.transform(train_data['review_without_punctuation'])
test_subsetX = vectorizer_subset.transform(test_data['review_without_punctuation'])

## Train a logistic regression model on a subset of data

Now build a logistic regression classifier with train_subsetX as features and sentiment as the target. Call this model simple_model .

In [101]:
simple_model = LogisticRegression().fit(train_subsetX, train_y)
simple_model

C:\Users\anupam\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [102]:
get_classification_accuracy(simple_model, test_subsetX, test_y)

0.87

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [127]:
sig_word_coeff = list(zip(significant_words, simple_model.coef_[0]))

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [126]:
simple_sorted_coeff = sorted(sig_word_coeff, key=lambda item:item[1], reverse=True)
x = [print(word, coeff) for (word, coeff) in simple_sorted_coeff]

loves 1.7002126297879914
perfect 1.5211189888958407
love 1.3565592559668063
easy 1.1910888644843223
great 0.9304893797198319
little 0.5016435568952592
well 0.4963916116267885
able 0.19415647830494415
car 0.07437784605925846
old 0.07200503919828553
less -0.2072725288874614
product -0.31254568418681616
would -0.33976004912745755
even -0.49098622834583944
work -0.6377412762768394
money -0.9390450134203616
broke -1.6674849670557026
waste -2.0078030433921525
return -2.0778697599619242
disappointed -2.3892080334806995


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [132]:
print_model_coeff(simple_model)

No. of coefficients: 20
No of +ve coefficients: 10
No of -ve coefficients: 10


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [137]:
positive_significant_words = [word for (word, coeff) in simple_sorted_coeff if coeff > 0]
all_words_coeff = list(zip(vectorizer.vocabulary_.keys(), sentiment_model.coef_[0]))
print('The coefficients of positive significant words in sentiment model are: ')
all_positive_words = [print(word, coeff) for (word, coeff) in all_words_coeff if word in positive_significant_words]

The coefficients of positive significant words in sentiment model are: 
little 0.15164453105926956
loves 0.011549433621536582
great 0.047411474236942606
able -1.3615587537135288e-05
love -0.5151738423059505
well 0.0007458305098735024
easy -0.32309924542536894
old -4.956880375287803e-06
perfect 0.011841798351930598
car 0.4449835489327556


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [138]:
get_classification_accuracy(sentiment_model, train_X, train_y)

0.97

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [139]:
get_classification_accuracy(simple_model, train_subsetX, train_y)

0.87

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [141]:
get_classification_accuracy(sentiment_model, test_X, test_y)

0.93

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [143]:
get_classification_accuracy(simple_model, test_subsetX, test_y)

0.87

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [145]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112182
21219


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [147]:
num_test_positive = (test_y == +1).sum()
num_test_negative = (test_y == -1).sum()
print(num_test_positive, num_test_negative)
majority_class_classifier_accuracy = round(num_test_positive / len(test_y), 2)
print('majority class classifier accuracy: {}'.format(majority_class_classifier_accuracy))

28077 5274
majority class classifier accuracy: 0.84


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?